In [110]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [41]:
df = pd.read_csv("starcraft_player_data.csv")
df.head()

,GameID,LeagueIndex,Age,HoursPerWeek,TotalHours,APM,SelectByHotkeys,AssignToHotkeys,UniqueHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilitiesUsed
0,52,5,27,10,3000,143.7180,0.003515,0.000220,7,0.000110,0.000392,0.004849,32.6677,40.8673,4.7508,28,0.001397,6,0.0,0.000000
1,55,5,23,10,5000,129.2322,0.003304,0.000259,4,0.000294,0.000432,0.004307,32.9194,42.3454,4.8434,22,0.001193,5,0.0,0.000208
2,56,4,30,10,200,69.9612,0.001101,0.000336,4,0.000294,0.000461,0.002926,44.6475,75.3548,4.0430,22,0.000745,6,0.0,0.000189
3,57,3,19,20,400,107.6016,0.001034,0.000213,1,0.000053,0.000543,0.003783,29.2203,53.7352,4.9155,19,0.000426,7,0.0,0.000384
4,58,3,32,10,500,122.8908,0.001136,0.000327,2,0.000000,0.001329,0.002368,22.6885,62.0813,9.3740,15,0.001174,4,0.0,0.000019


In [42]:
df.isna().mean()

GameID                  0.0
LeagueIndex             0.0
Age                     0.0
HoursPerWeek            0.0
TotalHours              0.0
APM                     0.0
SelectByHotkeys         0.0
AssignToHotkeys         0.0
UniqueHotkeys           0.0
MinimapAttacks          0.0
MinimapRightClicks      0.0
NumberOfPACs            0.0
GapBetweenPACs          0.0
ActionLatency           0.0
ActionsInPAC            0.0
TotalMapExplored        0.0
WorkersMade             0.0
UniqueUnitsMade         0.0
ComplexUnitsMade        0.0
ComplexAbilitiesUsed    0.0
dtype: float64

There are no null values in the dataset. However, this doesn't mean there isn't missingness in the dataset. It could be represented in other ways than just being a null value. This being said, there is no use for the GameID column while using this dataset to predict rank. It is an ID for each game, and won't have any correlation with rank.

In [43]:
df = df.drop(columns=['GameID'])
df.describe()

,LeagueIndex,APM,SelectByHotkeys,AssignToHotkeys,UniqueHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilitiesUsed
count,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000
mean,4.184094,117.046947,0.004299,0.000374,4.364654,0.000098,0.000387,0.003463,40.361562,63.739403,5.272988,22.131664,0.001032,6.534021,0.000059,0.000142
std,1.517327,51.945291,0.005284,0.000225,2.360333,0.000166,0.000377,0.000992,17.153570,19.238869,1.494835,7.431719,0.000519,1.857697,0.000111,0.000265
min,1.000000,22.059600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000679,6.666700,24.093600,2.038900,5.000000,0.000077,2.000000,0.000000,0.000000
25%,3.000000,79.900200,0.001258,0.000204,3.000000,0.000000,0.000140,0.002754,28.957750,50.446600,4.272850,17.000000,0.000683,5.000000,0.000000,0.000000
50%,4.000000,108.010200,0.002500,0.000353,4.000000,0.000040,0.000281,0.003395,36.723500,60.931800,5.095500,22.000000,0.000905,6.000000,0.000000,0.000020
75%,5.000000,142.790400,0.005133,0.000499,6.000000,0.000119,0.000514,0.004027,48.290500,73.681300,6.033600,27.000000,0.001259,8.000000,0.000086,0.000181
max,8.000000,389.831400,0.043088,0.001752,10.000000,0.003019,0.004041,0.007971,237.142900,176.372100,18.558100,58.000000,0.005149,13.000000,0.000902,0.003084


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3395 entries, 0 to 3394
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   LeagueIndex           3395 non-null   int64  
 1   Age                   3395 non-null   object 
 2   HoursPerWeek          3395 non-null   object 
 3   TotalHours            3395 non-null   object 
 4   APM                   3395 non-null   float64
 5   SelectByHotkeys       3395 non-null   float64
 6   AssignToHotkeys       3395 non-null   float64
 7   UniqueHotkeys         3395 non-null   int64  
 8   MinimapAttacks        3395 non-null   float64
 9   MinimapRightClicks    3395 non-null   float64
 10  NumberOfPACs          3395 non-null   float64
 11  GapBetweenPACs        3395 non-null   float64
 12  ActionLatency         3395 non-null   float64
 13  ActionsInPAC          3395 non-null   float64
 14  TotalMapExplored      3395 non-null   int64  
 15  WorkersMade          

In [45]:
df['Age'].unique()

array(['27', '23', '30', '19', '32', '21', '17', '20', '18', '16', '26',
       '38', '28', '25', '22', '29', '24', '35', '31', '33', '37', '40',
       '34', '43', '41', '36', '44', '39', '?'], dtype=object)

Although there are no null values in the dataset, they appear to be marked by '?.'

In [46]:
means = df.groupby('LeagueIndex').mean()
for col in df.columns:
    df[col] = df[col].replace('?', np.nan)

In [52]:
df.isna().mean()

LeagueIndex             0.000000
Age                     0.016200
HoursPerWeek            0.016495
TotalHours              0.016789
APM                     0.000000
SelectByHotkeys         0.000000
AssignToHotkeys         0.000000
UniqueHotkeys           0.000000
MinimapAttacks          0.000000
MinimapRightClicks      0.000000
NumberOfPACs            0.000000
GapBetweenPACs          0.000000
ActionLatency           0.000000
ActionsInPAC            0.000000
TotalMapExplored        0.000000
WorkersMade             0.000000
UniqueUnitsMade         0.000000
ComplexUnitsMade        0.000000
ComplexAbilitiesUsed    0.000000
dtype: float64

Because the null values make up such a small portion of the data (<2%), we can drop the null values and continue to training our model.

In [53]:
df = df.dropna()

In [54]:
df.corr()

,LeagueIndex,APM,SelectByHotkeys,AssignToHotkeys,UniqueHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilitiesUsed
LeagueIndex,1.000000,0.624171,0.428637,0.487280,0.322415,0.270526,0.206380,0.589193,-0.537536,-0.659940,0.140303,0.230347,0.310452,0.151933,0.171190,0.156033
APM,0.624171,1.000000,0.814624,0.534134,0.335986,0.218562,0.306392,0.635248,-0.567396,-0.722253,0.402928,0.239336,0.377719,0.122916,0.161774,0.141060
SelectByHotkeys,0.428637,0.814624,1.000000,0.450342,0.274030,0.132723,0.107615,0.360057,-0.273767,-0.390012,0.166964,0.097397,0.161404,0.027628,0.065467,0.063720
AssignToHotkeys,0.487280,0.534134,0.450342,1.000000,0.402555,0.205436,0.154998,0.454480,-0.377926,-0.461496,0.091508,0.201862,0.197011,0.149329,0.169849,0.169048
UniqueHotkeys,0.322415,0.335986,0.274030,0.402555,1.000000,0.151029,0.124570,0.353112,-0.223745,-0.304583,-0.022227,0.267940,0.111319,0.233274,0.122352,0.110004
MinimapAttacks,0.270526,0.218562,0.132723,0.205436,0.151029,1.000000,0.224683,0.137745,-0.213330,-0.171464,0.133712,0.164110,0.082283,0.125035,0.052225,0.042265
MinimapRightClicks,0.206380,0.306392,0.107615,0.154998,0.124570,0.224683,1.000000,0.143538,-0.244542,-0.216644,0.323707,0.173514,0.212464,0.154447,0.097992,0.095659
NumberOfPACs,0.589193,0.635248,0.360057,0.454480,0.353112,0.137745,0.143538,1.000000,-0.491407,-0.817162,-0.242571,0.470955,0.282204,0.322181,0.196269,0.177696
GapBetweenPACs,-0.537536,-0.567396,-0.273767,-0.377926,-0.223745,-0.213330,-0.244542,-0.491407,1.000000,0.680483,-0.310242,-0.094693,-0.237157,-0.089708,-0.082924,-0.092005
ActionLatency,-0.659940,-0.722253,-0.390012,-0.461496,-0.304583,-0.171464,-0.216644,-0.817162,0.680483,1.000000,-0.106505,-0.349350,-0.313692,-0.223191,-0.198054,-0.189681


The LeagueIndex column, which we are trying to predict, is most highly correlated with the following columns:
- APM
- AssignToHotkeys 
- NumberOfPACs 
- GapBetweenPACs
- ActionLatency 

Other notably high correlations include:
- APM x SelectByHotkeys
- NumberOfPACs x Action Latency

Let's start with our model. First, I'll split the data into train and test data. We'll do 2 sets of these, one for the highly correlated variables, and one for all of them.

In [67]:
X = df[['Age', 'HoursPerWeek', 'TotalHours', 'APM',
       'SelectByHotkeys', 'AssignToHotkeys', 'UniqueHotkeys', 'MinimapAttacks',
       'MinimapRightClicks', 'NumberOfPACs', 'GapBetweenPACs', 'ActionLatency',
       'ActionsInPAC', 'TotalMapExplored', 'WorkersMade', 'UniqueUnitsMade',
       'ComplexUnitsMade', 'ComplexAbilitiesUsed']]
y = df['LeagueIndex']

X_corr = df[['APM', 'AssignToHotkeys', 'NumberOfPACs', 'GapBetweenPACs', 'ActionLatency']]

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_corr, y, test_size=0.2)

Let's also define an accuracy function

In [77]:
def acc(model, X, y):
    return (model.predict(X) == y).mean()

Next, we'll be training a model and seeing how well it works. We'll start with the KNN algorithm. For simplicity, I'll be using sklearn's algorithm.

In [83]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [85]:
print(acc(knn, X_train, y_train))
print(acc(knn, X_test, y_test))

/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:230: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


0.547191011235955
0.35778443113772457


/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:230: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


The accuracy of KNN is extremely low. Let's try it with just the highly correlated variables with LeagueIndex and see if the accuracy increases. Again, those are:
- APM
- AssignToHotkeys 
- NumberOfPACs 
- GapBetweenPACs
- ActionLatency 

In [86]:
knn1 = KNeighborsClassifier(n_neighbors=5)
knn1.fit(X_train_1, y_train_1)
print(acc(knn1, X_train_1, y_train_1))
print(acc(knn1, X_test_1, y_test_1))


0.5468164794007491
0.3008982035928144


/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:230: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/dsc80/lib/python3.8/site-pack

The results are worse, from now, we'll be using all the variables. We'll be trying the following models and see which one best works with the given data:
- Logistic Regression
- Naive Bayes
- Decision Tree
- Random Forest

In [72]:
lr = LogisticRegression(multi_class='ovr', solver='liblinear')
lr.fit(X_train, y_train)

LogisticRegression(multi_class='ovr', solver='liblinear')

In [89]:
print(acc(lr, X_train, y_train))
print(acc(lr, X_test, y_test))

0.41048689138576777
0.344311377245509


Onto the Naive Bayes classifier

In [92]:
nb = GaussianNB()
nb.fit(X_train, y_train)

GaussianNB()

In [94]:
print(acc(nb, X_train, y_train))
print(acc(nb, X_test, y_test))

0.3767790262172285
0.34880239520958084


Onto the decision tree

In [96]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [97]:
print(acc(dt, X_train, y_train))
print(acc(dt, X_test, y_test))

1.0
0.33532934131736525


Looks like this model overfit, let's try to fix that by controlling the max depth.

In [108]:
dt1 = DecisionTreeClassifier(max_depth=8)
dt1.fit(X_train, y_train)
print(acc(dt1, X_train, y_train))
print(acc(dt1, X_test, y_test))

0.602996254681648
0.38023952095808383


Although, cross validation is best practice, I've always found testing by hand to be much more efficient. After testing, a depth of 8 is the best. Although the test accuracy isn't great, it's the best so far. Onto the random forest now.

In [131]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print(acc(rf, X_train, y_train))
print(acc(rf, X_test, y_test))

1.0
0.4236526946107784


The best test accuracy so far, let's see if we can optimize this by playing with the max depth for the classifier.

In [137]:
rf1 = RandomForestClassifier(max_depth=11)
rf1.fit(X_train, y_train)
print(acc(rf1, X_train, y_train))
print(acc(rf1, X_test, y_test))

0.9794007490636704
0.4086826347305389


The randomness of random forests makes the results slightly unreliable. Let's test and find an average of the test accuracy.

In [ ]:
control = 0
acc1 = 0
for i in range(5):
    rf.fit(X_train, y_train)
    rf1.fit(X_train, y_train)